# Lecture 3 — Support Triage + API Actions

**Goal**: Build a pipeline that turns support tickets into:

- triage labels (priority, category, route)
- a grounded draft reply with citations to a KB
- an optional action plan mapped to allow-listed API calls
- an actions log (what we executed, what we refused, why)

## Setup
Required env var:
- `OPENROUTER_API_KEY`

Optional:
- `OPENROUTER_MODEL` (default below)

We use local stub APIs (no network side effects).


In [ ]:
import json
import os
from pathlib import Path
from typing import Any, Dict, List

import httpx
import pandas as pd

DATA_DIR = Path("../data")
KB_DIR = DATA_DIR / "kb"
OUTPUT_DIR = DATA_DIR / "outputs"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
OPENROUTER_MODEL = os.getenv("OPENROUTER_MODEL", "openai/gpt-4o-mini")

if not OPENROUTER_API_KEY:
    raise RuntimeError("Missing OPENROUTER_API_KEY")


def openrouter_chat(messages: List[Dict[str, str]], *, temperature: float = 0.2) -> str:
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json",
    }
    payload = {"model": OPENROUTER_MODEL, "messages": messages, "temperature": temperature}
    with httpx.Client(timeout=60) as client:
        r = client.post(url, headers=headers, json=payload)
        r.raise_for_status()
        data = r.json()
    return data["choices"][0]["message"]["content"]


def parse_json(text: str) -> Dict[str, Any]:
    try:
        return json.loads(text)
    except json.JSONDecodeError as e:
        raise ValueError(f"Invalid JSON: {e}\n---\n{text}")


# Load tickets

tickets = [json.loads(line) for line in (DATA_DIR / "tickets.jsonl").read_text().splitlines() if line.strip()]
len(tickets), tickets[0]["ticket_id"]



In [ ]:
# KB retrieval (simple search)

kb_files = sorted(KB_DIR.glob("*.md"))
kb_texts = {p.name: p.read_text() for p in kb_files}


def retrieve_kb_snippets(query: str, *, top_k: int = 2) -> List[Dict[str, str]]:
    """Very simple retrieval: score by keyword overlap."""
    q = set(w.lower().strip(".,!?\"'()") for w in query.split())
    scored = []
    for name, text in kb_texts.items():
        t = set(w.lower().strip(".,!?\"'()") for w in text.split())
        score = len(q.intersection(t))
        scored.append((score, name, text))
    scored.sort(reverse=True)
    out = []
    for score, name, text in scored[:top_k]:
        out.append({"source": name, "snippet": "\n".join(text.splitlines()[:20])})
    return out


retrieve_kb_snippets("invalid API key in notebook")



In [ ]:
# End-to-end pipeline

TRIAGE_SCHEMA = {
    "priority": "low|medium|high",
    "category": "billing|bug|howto|security|integration|other",
    "route": "support|billing|engineering|security",
    "rationale": "string"
}

REPLY_SCHEMA = {
    "reply": "string",
    "citations": [{"source": "string", "quote": "string"}],
    "next_questions": ["string"],
}

ACTIONS_SCHEMA = {
    "actions": [
        {
            "type": "create_issue|request_refund|add_crm_note|none",
            "payload": "object"
        }
    ]
}

ALLOWED_ACTIONS = {"create_issue", "request_refund", "add_crm_note", "none"}

# Local API stubs
stub = json.loads((DATA_DIR / "api_stub_data.json").read_text())
customers = {c["customer_id"]: c for c in stub["customers"]}
invoices = {i["invoice_id"]: i for i in stub["invoices"]}


def stub_create_issue(payload: Dict[str, Any]) -> Dict[str, Any]:
    return {"status": "created", "issue_id": "ISSUE-" + payload.get("ticket_id", "UNK")}


def stub_request_refund(payload: Dict[str, Any]) -> Dict[str, Any]:
    invoice_id = payload.get("invoice_id")
    if invoice_id not in invoices:
        return {"status": "rejected", "reason": "unknown invoice"}
    return {"status": "requested", "invoice_id": invoice_id, "amount_usd": invoices[invoice_id]["amount_usd"]}


def stub_add_crm_note(payload: Dict[str, Any]) -> Dict[str, Any]:
    cid = payload.get("customer_id")
    if cid not in customers:
        return {"status": "rejected", "reason": "unknown customer"}
    return {"status": "ok", "customer_id": cid}


def triage_ticket(t: Dict[str, Any]) -> Dict[str, Any]:
    system = "Return ONLY valid JSON. No markdown."
    prompt = f"""Triage this support ticket.

JSON schema:
{json.dumps(TRIAGE_SCHEMA, indent=2)}

Ticket:
{json.dumps(t, indent=2)}

Guidance:
- billing issues -> route=billing
- suspected bug or crash -> route=engineering
- security/data handling -> route=security
"""
    text = openrouter_chat([
        {"role": "system", "content": system},
        {"role": "user", "content": prompt},
    ], temperature=0.0)
    return parse_json(text)


def draft_reply(t: Dict[str, Any], kb: List[Dict[str, str]]) -> Dict[str, Any]:
    system = "Return ONLY valid JSON. No markdown."
    prompt = f"""Write a helpful support reply.

You MUST cite KB quotes in `citations`.
Only use information from the KB snippets below.
If information is missing, ask clarifying questions.

JSON schema:
{json.dumps(REPLY_SCHEMA, indent=2)}

Ticket:
{json.dumps(t, indent=2)}

KB snippets:
{json.dumps(kb, indent=2)}
"""
    text = openrouter_chat([
        {"role": "system", "content": system},
        {"role": "user", "content": prompt},
    ], temperature=0.2)
    return parse_json(text)


def propose_actions(t: Dict[str, Any], triage: Dict[str, Any]) -> Dict[str, Any]:
    system = "Return ONLY valid JSON. No markdown."
    prompt = f"""Propose an action plan using ONLY these action types: {sorted(ALLOWED_ACTIONS)}.

JSON schema:
{json.dumps(ACTIONS_SCHEMA, indent=2)}

Ticket:
{json.dumps(t, indent=2)}

Triage:
{json.dumps(triage, indent=2)}

Rules:
- For duplicate charges, use request_refund with payload {{"invoice_id": "..."}} if present.
- For crashes/bugs, use create_issue with payload {{"ticket_id": "...", "summary": "..."}}.
- For security inquiries, use add_crm_note to flag follow-up.
- Otherwise, use none.
"""
    text = openrouter_chat([
        {"role": "system", "content": system},
        {"role": "user", "content": prompt},
    ], temperature=0.0)
    return parse_json(text)


def safe_execute_actions(t: Dict[str, Any], actions_obj: Dict[str, Any]) -> List[Dict[str, Any]]:
    logs: List[Dict[str, Any]] = []
    for a in actions_obj.get("actions", []):
        typ = a.get("type")
        payload = a.get("payload")

        if typ not in ALLOWED_ACTIONS:
            logs.append({"ticket_id": t["ticket_id"], "action": typ, "status": "refused", "reason": "not allow-listed"})
            continue
        if typ == "none":
            logs.append({"ticket_id": t["ticket_id"], "action": "none", "status": "skipped"})
            continue
        if not isinstance(payload, dict):
            logs.append({"ticket_id": t["ticket_id"], "action": typ, "status": "refused", "reason": "payload must be object"})
            continue

        if typ == "create_issue":
            res = stub_create_issue({**payload, "ticket_id": t["ticket_id"]})
        elif typ == "request_refund":
            res = stub_request_refund(payload)
        elif typ == "add_crm_note":
            res = stub_add_crm_note({**payload, "customer_id": t["customer_id"]})
        else:
            res = {"status": "unknown"}

        logs.append({"ticket_id": t["ticket_id"], "action": typ, "status": res.get("status"), "result": json.dumps(res)})
    return logs


triage_rows = []
reply_rows = []
action_logs = []

for t in tickets:
    tri = triage_ticket(t)
    kb = retrieve_kb_snippets(t["text"], top_k=2)
    rep = draft_reply(t, kb)
    acts = propose_actions(t, tri)
    logs = safe_execute_actions(t, acts)

    triage_rows.append({"ticket_id": t["ticket_id"], **tri})
    reply_rows.append({"ticket_id": t["ticket_id"], "reply": rep.get("reply", ""), "citations": json.dumps(rep.get("citations", []))})
    action_logs.extend(logs)

triage_df = pd.DataFrame(triage_rows)
replies_df = pd.DataFrame(reply_rows)
actions_df = pd.DataFrame(action_logs)

triage_df.to_csv(OUTPUT_DIR / "triage.csv", index=False)
replies_df.to_csv(OUTPUT_DIR / "draft_replies.csv", index=False)
actions_df.to_csv(OUTPUT_DIR / "actions_log.csv", index=False)

triage_df, replies_df.head(2), actions_df



## Extensions / Optional challenges

- **Stronger grounding**: require exact quotes in citations; reject replies that reference uncited facts.
- **Better retrieval**: upgrade keyword overlap to embeddings or hybrid; measure impact on reply quality.
- **Ticket state machine**: enforce allowed transitions (new → triaged → awaiting_customer → resolved).
- **Safer execution**: per-action schema validation + dry-run mode + retry/backoff + richer logs.
- **Adversarial tickets**: include prompt injection in ticket text; harden with instruction hierarchy and allow-lists.
